In [52]:
from random import random
from abc import ABC, abstractmethod
from enum import Enum

class Strategy(Enum):
    ALWAYS_HELP = 1
    TIT_FOR_TAT = 2
    ALWAYS_SABOTAGE = 3

class Action(Enum):
    HELP = 1
    SABOTAGE = 2

class BiMatrix:
    def __init__(self, bi_matrix):
        self.bi_matrix = bi_matrix
    
    def get_row_output(self, column_input):
        pass  

    def get_column_output(self, row_input):
        pass

    def get_outcome(self, row_input: Action, column_input: Action):
        if row_input == Action.HELP:
            row = self.bi_matrix[0]
        elif row_input == Action.SABOTAGE:
            row = self.bi_matrix[1]

        if column_input == Action.HELP:
            payoff = row[0]
        elif column_input == Action.SABOTAGE:
            payoff = row[1]

        return payoff[0] # Can only do it like this since bi-matrix is symmetrical

class Contestant(ABC):
    def __init__(self, talent_idx, id):
        self.talent = talent_idx
        self.id = id
        self.no_of_votes = 0
        self.vote_factor = None
        self.bi_matrix = BiMatrix([
            [(1.05, 1.05), (0.9, 1.1)],
            [(1.1, 0.9), (0.85, 0.85)]
        ])
        self.previous_action = None

    def simulate_voting(self, no_of_voters = 10000):
        self.restart_votes()
        
        for _ in range(no_of_voters):
            if random() < self.talent:
                self.no_of_votes += 1

        self.no_of_votes = round(self.no_of_votes * self.vote_factor)

    def restart_votes(self):
        self.no_of_votes = 0

    @abstractmethod
    def determine_action(self, opponent):
        pass
        
class ConstantStrategyContestant(Contestant):
    def __init__(self, talent_idx, id):
        super().__init__(talent_idx, id)

        self.strategy = self.initialise_strategy()

    def initialise_strategy(self):
        if self.talent < 0.5:
            strategy = Strategy.ALWAYS_SABOTAGE
        elif 0.5 <= self.talent < 0.8:
            strategy = Strategy.TIT_FOR_TAT
        else:
            strategy = Strategy.ALWAYS_HELP

        return strategy
    
    def determine_action(self, opponent):
        if self.strategy == Strategy.TIT_FOR_TAT:
            if opponent.previous_action is None:
                action = Action.HELP
            else:
                action = opponent.previous_action
        elif self.strategy == Strategy.ALWAYS_HELP:
            action = Action.HELP
        elif self.strategy == Strategy.ALWAYS_SABOTAGE:
            action = Action.SABOTAGE
            
        self.previous_action = action

        return action
    
    def determine_vote_factor(self, opponent_action: Action):
        self.vote_factor = self.bi_matrix.get_outcome(self.previous_action, opponent_action)
        

In [ ]:
import numpy as np

"""Set up season"""

# Create contestants with different talent levels
def create_strategy_contestants(talent_list: list):
    contestants = []
    for i in range(len(talent_list)):
        contestants.append(ConstantStrategyContestant(talent_list[i], i))
    return contestants

# Simulate interactions between contestants
def simulate_interactions(contestants: list[Contestant]):
    np.random.shuffle(contestants)

    # Group contestants in pairs
    groups = []
    i = 0
    while i < len(contestants):
        groups.append([contestants[i], contestants[i + 1]])
        i += 2
    
    # Determine vote factor from interaction
    for group in groups:
        contestant_1 = group[0]
        contestant_2 = group[1]

        action_1 = contestant_1.determine_action(contestant_2)
        action_2 = contestant_2.determine_action(contestant_1)

        contestant_1.determine_vote_factor(action_2)
        contestant_2.determine_vote_factor(action_1)

    return contestants
        

def simulate_season(contestants, no_of_voters=10000, episodes=10, elimination_episodes=[3, 6, 10], safe_zone=[16, 10, 5]):
    for episode in range(episodes):
        print(f"Episode {episode + 1}")

        # Group contestants in pairs and determine voting factor from interaction
        contestants = simulate_interactions(contestants)
        
        # Simulate voting for each contestant
        for contestant in contestants:
            contestant.simulate_voting(no_of_voters)
        
        # Eliminate contestants if it's an elimination episode
        if episode + 1 in elimination_episodes:
            contestants.sort(key=lambda x: x.no_of_votes, reverse=True)
            
            # Eliminate all contestants who are not on or above the safe zone (i.e. in the elimination zone)
            while len(contestants) > safe_zone[elimination_episodes.index(episode + 1)]:
                contestants.pop()
    
    print("Season simulation complete.\n")

    # Print debut group
    print("Debut group:")
    for contestant in contestants:
        print(f"Contestant {contestant.id} with talent {contestant.talent:.2f} and votes {contestant.no_of_votes}")

uniform_distribution_talents = [
    0.03, 0.07, 0.11, 0.13, 0.16, 0.18, 0.21, 0.23, 0.26, 0.29,
    0.31, 0.34, 0.36, 0.39, 0.41, 0.44, 0.46, 0.49, 0.53, 0.56,
    0.59, 0.62, 0.65, 0.68, 0.71, 0.74, 0.78, 0.82, 0.89, 0.95
]
normal_distribution_talents = [
    0.12, 0.18, 0.22, 0.27, 0.31, 0.34, 0.37, 0.39, 0.41, 0.43,
    0.45, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.55, 0.57,
    0.59, 0.61, 0.63, 0.66, 0.69, 0.73, 0.77, 0.82, 0.88, 0.94
]
many_high_talents = [
    0.35, 0.42, 0.55, 0.62, 0.68, 0.72, 0.76, 0.78, 0.79, 0.8,
    0.8, 0.8, 0.8, 0.81, 0.81, 0.81, 0.82, 0.82, 0.82, 0.83,
    0.83, 0.84, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.92, 0.95
]

contestants = create_strategy_contestants(normal_distribution_talents)
simulate_season(contestants)


Episode 1
Episode 2
Episode 3
Episode 4
Episode 5
Episode 6
Episode 7
Episode 8
Episode 9
Episode 10
Season simulation complete.

Debut group:
Contestant 29 with talent 0.95 and votes 9991
Contestant 28 with talent 0.89 and votes 9394
Contestant 27 with talent 0.82 and votes 8633
Contestant 26 with talent 0.78 and votes 8104
Contestant 25 with talent 0.74 and votes 7727
